In [51]:
# Description: FHIR 서버와 연결하기 위한 코드 . hskim

#Imports
from fhirpy import SyncFHIRClient # fhir 표준을 사용하기 위한 라이브러리
from fhir.resources.patient import Patient # fhir 리소스 중 환자 리소스를 사용하기 위한 라이브러리
from fhir.resources.observation import Observation # fhir 리소스 중 Observation 리소스를 사용하기 위한 라이브러리, Lab Test 결과를 저장하기 위해 사용
from fhir.resources.humanname import HumanName # fhir 리소스 중 HumanName 리소스를 사용하기 위한 라이브러리, 환자 이름을 저장하기 위해 사용
from fhir.resources.contactpoint import ContactPoint # fhir 리소스 중 ContactPoint 리소스를 사용하기 위한 라이브러리, 환자 연락처를 저장하기 위해 사용
from fhir.resources.reference import Reference # fhir 리소스 중 Reference 리소스를 사용하기 위한 라이브러리, 환자 정보를 저장하기 위해 사용
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding 
from fhir.resources.quantity import Quantity 

from fhirpy.base.exceptions import ResourceNotFound # fhirpy exception을 처리하기 위한 라이브러리

import subprocess
import json

# import hs_config as CONFIG # 개인 config 내용을 정리하기 위한 파일

BASE_URL = "http://127.0.0.1:5000" # "https://hapi.fhir.org/baseR4"  # FHIR 서버의 URL 혹은 ip와 연결
TOKEN_URL = "YOUR_TOKEN_URL"
    
class FHIRClient(): # Fhir 서버와 연결하기 위한 클래스
    def __init__(self) -> None:
        self.m_client = None
        self.m_access_token = None
        self.m_refresh_token = None
        # self.RequestAccessToken()
        self.CreateClient()

    def RequestAccessToken(self):
        # 이 CURL Command 내에서 서버에서 받아와야하는 certification 요청
        # Command의 내용은 서버에서 받아와야 하는 내용에 따라 변경
        curl_command = f"curl --location {TOKEN_URL} \
        --header 'Content-Type: application/x-www-form-urlencoded'" 

        # curl 명령 실행
        process = subprocess.Popen(curl_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
        response, error = process.communicate()

        # 응답 처리
        if process.returncode == 0:
            response_data = json.loads(response.decode('utf-8'))
            # 특정 키워드 값 추출
            self.m_access_token = response_data.get('access_token', None) # 받아와야 하는 토큰의 이름에 따라 acceess_token 혹은 refresh_toke 등으로 변경
        else:
            print(f"Error: {error.decode('utf-8')}")

    def CreateClient(self):
        # FHIR 클라이언트 인스턴스 생성
        self.m_client = SyncFHIRClient(BASE_URL) # 본인이 연결하고자 하는 서버의 URL 혹은 ip와 연결
            #, authorization=f'Bearer {self.m_access_token}'  # Bearer 토큰 인증
            # 토큰이 필요하지 않은 경우 authorization 부분은 삭제

        # 서버와의 연결 확인
        try:
            # 서버에서 임의의 리소스를 가져와 연결 확인
            # self.m_client.resources('Patient').fetch() # 자체 서버인 경우 데이터가 없을 때 문제 생기니까 주석처리해야함
            print("서버와의 연결이 확인되었습니다.")
        except ResourceNotFound:
            print("서버에 리소스가 없거나 접근할 수 없습니다.")
        except Exception as e:
            print(f"서버와의 연결 확인 중 오류 발생: {e}")

client = FHIRClient()

서버와의 연결이 확인되었습니다.


### 환자 저장
dict 타입으로 저장하는 방법

In [52]:
# 가상의 환자 p
p = {
    'patient_number': 1234567890,
    'family_name': '홍',
    'given_name': '길동',
    'sex' : 'F'
}

# 더미 환자 데이터 추가
new_patient = client.m_client.resource(
    'Patient',
    identifier=[{
    'system': 'http://YOUR.SYSTEM.URL/',
    'value': str(p['patient_number'])
}],
    name=[{
        'use': 'official',
        'family': str(p['family_name']),
        'given': [str(p['given_name'])]
    }],
    gender = "female" if p['sex'] == "F" else "male"
    #,id = str(p['patient_number'])
    )

# 환자 데이터 저장
new_patient.save()

### 환자 검색

In [53]:
search = client.m_client.resources('Patient').search(identifier=p['patient_number'])
patients = search.fetch()

for pt in patients:
    print(pt.name[0])

{'use': 'official', 'family': '홍', 'given': ['길동']}


### Observation 저장
클래스 개별 선언으로 저장하는 방법

In [54]:
# 시스템 내에서 사용하는 코드, 코드명, 코드 설명을 설정
coding = Coding()
coding.system = 'http://YOUR.SYSTEM.URL/'
coding.code = "2093-3"
coding.display = "Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma"
code = CodeableConcept()
code.coding = [coding]
code.text = "Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma"

#fhir.resources observation 생성
observation0 = Observation(status="final",code=code)

category = CodeableConcept()
category.coding = [coding]

observation0.category = [category]
observation0.effectiveDateTime = "2012-05-10T11:59:49+00:00"
observation0.issued = "2012-05-10T11:59:49.565+00:00"

# 코드, 단위, 시스템, 값으로 구성된 valueQuantity 설정
valueQuantity = Quantity()
valueQuantity.code = "x"
valueQuantity.unit = "x"
valueQuantity.system = "https://unitsofmeasure.org"
valueQuantity.value = 37.395

observation0.valueQuantity = valueQuantity

# 레퍼런스 세팅
id = p['patient_number']
reference = Reference()
reference.reference = f"Patient/{id}"
reference.id = str(id)

observation0.subject = reference

ob = json.loads(observation0.json())

client.m_client.resource('Observation',**ob).save()

### Observation 검색

In [55]:
search = client.m_client.resources('Observation').search(id=p['patient_number'])
observation = search.fetch()

print(len(observation))
for ob in observation:
    print(ob.category)

1
[{'coding': [{'system': 'http://YOUR.SYSTEM.URL/', 'code': '2093-3', 'display': 'Aspartate aminotransferase [Enzymatic activity/volume] in Serum or Plasma'}]}]
